In [2]:
from pyspark.sql import SparkSession

# create spark session
spark = SparkSession.builder \
    .appName('dataprocessing') \
    .getOrCreate()

file_path = 'data/report_events_TRAN00019_202401_202401.csv'

sdf = spark.read.format('csv').option('header', 'true').load(file_path)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 08:30:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
# count no of rows
sdf.count() 

41831590

In [23]:
# show timestamp column head
sdf.select('event_ts').show(n=5, truncate=False)

+-------------------+
|event_ts           |
+-------------------+
|2024-01-01 04:26:57|
|2024-01-01 04:27:07|
|2024-01-01 12:54:57|
|2024-01-01 19:09:29|
|2024-01-01 19:09:30|
+-------------------+
only showing top 5 rows



In [22]:
# show column datatype
sdf.select('event_ts').dtypes

[('event_ts', 'timestamp')]

In [18]:
# check for null
sdf.filter(sdf.event_ts.isNull()).collect()

[]

In [21]:
from pyspark.sql.functions import to_timestamp, col

# convert string to timestamp
sdf = sdf.withColumn("event_ts", to_timestamp(col("event_ts")))


In [26]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

windowSpec = Window.orderBy("event_ts")

# time difference, belom bisa
sdf = sdf.withColumn("prev_event_ts", lag("event_ts").over(windowSpec))
sdf = sdf.withColumn("time_diff", col("event_ts") - col("prev_event_ts"))

sdf.select('event_ts', 'prev_event_ts', 'time_diff').show(n=5, truncate=False)

24/08/07 09:35:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/07 09:35:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/07 09:35:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/07 09:35:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/07 09:35:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+-------------+---------+
|event_ts|prev_event_ts|time_diff|
+--------+-------------+---------+
|null    |null         |null     |
|null    |null         |null     |
|null    |null         |null     |
|null    |null         |null     |
|null    |null         |null     |
+--------+-------------+---------+
only showing top 5 rows

